In [1]:
!pip install datasets
!pip install transformers --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requ

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Use a pipeline as a high-level helper
from transformers import pipeline

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NepBERTa/NepBERTa")
model = AutoModelForSequenceClassification.from_pretrained("NepBERTa/NepBERTa",from_tf=True).to(device)

classifier = pipeline("sentiment-analysis", model=model, tokenizer = tokenizer)

cuda


config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tf_model.h5:   0%|          | 0.00/534M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
import pandas as pd
#load cleaned train set
df_train = pd.read_csv('/content/drive/MyDrive/SynonymReplacement/cleaned_train.csv')
#load the cleaned test set
df_test = pd.read_csv('/content/drive/MyDrive/NepaliProject1/Augmented/cleaned_test.csv')

In [6]:
#make a dataset which is the original train set repeated 6 times
copy1 = df_train.copy()
copy2 = df_train.copy()
copy3 = df_train.copy()
copy4 = df_train.copy()
copy5 = df_train.copy()
df_train_times6 = pd.concat([df_train,copy1,copy2,copy3,copy4,copy5])

In [7]:
print( df_train_times6.shape)

(35940, 3)


In [13]:
from datasets import Dataset, DatasetDict

df_trainset = Dataset.from_pandas(df_train_times6)
test_dataset = Dataset.from_pandas(df_test)

final_dataset = DatasetDict({
    'train':df_trainset,
    'test': test_dataset,
})


In [14]:
def tokenize_texts(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')

In [15]:
encoded_data = final_dataset.map(tokenize_texts, batched=True, batch_size=None)

Map:   0%|          | 0/35940 [00:00<?, ? examples/s]

Map:   0%|          | 0/1993 [00:00<?, ? examples/s]

In [16]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 35940
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1993
    })
})

In [25]:
def extract_hidden_states(batch):
  #inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
   # Convert lists to tensors before moving to device
  inputs = {k: torch.tensor(v).to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
  # Extract last hidden states
  with torch.no_grad():
      last_hidden_state = model(**inputs).logits
          # Return vector for [CLS] token
      return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [26]:

text = "असाध्यै राम्रो कार्यक्रम आयोजना गरिएको छ."

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt").to(device)

# Forward pass through the model
outputs = model(**inputs)

# Access the last hidden states
last_hidden_states = outputs.logits


In [27]:
encoded_data = encoded_data.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/35940 [00:00<?, ? examples/s]

Map:   0%|          | 0/1993 [00:00<?, ? examples/s]

In [28]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = (AutoModelForSequenceClassification
         .from_pretrained("NepBERTa/NepBERTa", num_labels=num_labels, from_tf=True)
         .to(device))

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [29]:

from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [30]:
from transformers import Trainer, TrainingArguments

#main path
main_path = '/content/drive/MyDrive/Nepaliproject2/6times_original_dataset'

batch_size = 64
logging_steps = len(encoded_data["train"])
model_name = main_path+"/model"
training_args = TrainingArguments(output_dir='/content/drive/MyDrive/Nepaliproject2/6times_original_dataset',
                                  num_train_epochs=1,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:

trainer = Trainer(model=model, args=training_args,
                      compute_metrics=compute_metrics,train_dataset=encoded_data["train"],
                      eval_dataset=encoded_data["test"],tokenizer=tokenizer)
trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.885584,0.656297,0.653755


In [32]:
preds_output = trainer.predict(encoded_data["test"])

In [33]:
preds_output.metrics

{'test_loss': 0.8855841755867004,
 'test_accuracy': 0.6562970396387355,
 'test_f1': 0.6537554960580093,
 'test_runtime': 13.7846,
 'test_samples_per_second': 144.582,
 'test_steps_per_second': 2.321}

In [34]:
trainer.save_model('/content/drive/MyDrive/Nepaliproject2/6times_original_dataset/TrainedModel')